In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('DataSets\SuperStoreOrders.csv')

<>:1: SyntaxWarning: invalid escape sequence '\S'
<>:1: SyntaxWarning: invalid escape sequence '\S'
C:\Users\amand\AppData\Local\Temp\ipykernel_3424\1137792038.py:1: SyntaxWarning: invalid escape sequence '\S'
  df = pd.read_csv('DataSets\SuperStoreOrders.csv')


In [6]:
df.dtypes

order_id           object
order_date         object
ship_date          object
ship_mode          object
customer_name      object
segment            object
state              object
country            object
market             object
region             object
product_id         object
category           object
sub_category       object
product_name       object
sales              object
quantity            int64
discount          float64
profit            float64
shipping_cost     float64
order_priority     object
year                int64
dtype: object

In [7]:
#Since we will insert this data into SQL, so it is important to change the datatypes
df['sales'] = df['sales'].str.replace(',','').astype(int)
df['order_date'] = pd.to_datetime(df['order_date'], dayfirst = True)
df['ship_date'] = pd.to_datetime(df['ship_date'], dayfirst = True)


In [8]:
df.dtypes

order_id                  object
order_date        datetime64[ns]
ship_date         datetime64[ns]
ship_mode                 object
customer_name             object
segment                   object
state                     object
country                   object
market                    object
region                    object
product_id                object
category                  object
sub_category              object
product_name              object
sales                      int64
quantity                   int64
discount                 float64
profit                   float64
shipping_cost            float64
order_priority            object
year                       int64
dtype: object

In [9]:
df.columns

Index(['order_id', 'order_date', 'ship_date', 'ship_mode', 'customer_name',
       'segment', 'state', 'country', 'market', 'region', 'product_id',
       'category', 'sub_category', 'product_name', 'sales', 'quantity',
       'discount', 'profit', 'shipping_cost', 'order_priority', 'year'],
      dtype='object')

In [10]:
df.columns = df.columns.str.replace('_', ' ')

In [11]:
df.columns = df.columns.str.title()

In [12]:
df.columns

Index(['Order Id', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer Name',
       'Segment', 'State', 'Country', 'Market', 'Region', 'Product Id',
       'Category', 'Sub Category', 'Product Name', 'Sales', 'Quantity',
       'Discount', 'Profit', 'Shipping Cost', 'Order Priority', 'Year'],
      dtype='object')

In [13]:
df.shape

(51290, 21)

In [14]:
df.isnull().sum().sum() #No Null Values

np.int64(0)

In [ ]:
unique_customers = df[['Customer Name', 'Segment', 'Country',
                        'State', 'Region', 'Market']].drop_duplicates().reset_index(drop=True)


# Creating Customer_ID as Primary Key For SQL
unique_customers['Customer Id'] = unique_customers.index + 1

# Merging Customer_ID Back To The Original DataFrame
df = df.merge(unique_customers, on=['Customer Name', 'Segment', 'Country', 'State', 'Region', 'Market'], how='left')


In [ ]:
#Partioning The Main Data Into Three Tables -> Customers, Products & Orders
customers = df[['Customer Id', 'Customer Name', 'Segment', 'Country',
                 'State', 'Region', 'Market']].drop_duplicates().reset_index(drop=True)

products = df[['Product Id', 'Product Name', 'Category',
                'Sub Category']].drop_duplicates().reset_index(drop=True)

orders = df[['Order Id', 'Order Date', 'Ship Date', 'Ship Mode','Customer Id', 'Product Id', 'Sales', 'Quantity','Discount', 'Profit', 'Shipping Cost', 'Order Priority', 'Year']]


In [17]:
customers.to_csv("customers.csv", index=False)
products.to_csv("products.csv", index=False)
orders.to_csv("orders.csv", index=False)


In [18]:
df_products = pd.read_csv('products.csv')

In [19]:
df_products.columns

Index(['Product Id', 'Product Name', 'Category', 'Sub Category'], dtype='object')

In [20]:
df_products = df_products.drop_duplicates(subset = 'Product Id')

In [21]:
df_products.to_csv('products_clean.csv', index = False)

In [22]:
df_orders = pd.read_csv('orders.csv')

In [23]:
df_orders.columns

Index(['Order Id', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer Id',
       'Product Id', 'Sales', 'Quantity', 'Discount', 'Profit',
       'Shipping Cost', 'Order Priority', 'Year'],
      dtype='object')

In [24]:
df_orders = df_orders.drop_duplicates(subset = ['Order Id', 'Product Id'])

In [25]:
df_orders.to_csv('orders_clean.csv', index = False)